In [1]:
import pickle
from pympler import asizeof
import time
from tqdm import tqdm_notebook

In [2]:
posting=pickle.load(open("backup/posting.p","rb"))
docId=pickle.load(open("backup/docId.p","rb"))
time_to_index=pickle.load(open("backup/time_to_index.p","rb"))

In [3]:
def compress():
    longstr = "";
    ptr = {}
    posts = []
    i = 0
    num = 0
    for key in tqdm_notebook(posting):
        longstr += key

        post = posting[key][1]
        temp_post = []
        for docId in range(len(post)):
            if(docId == 0):
                temp_post.append(post[0])
            else:
                temp_post.append(post[docId]-post[docId-1])
        posts.append(encode(temp_post))

        ptr[num] = i
        i+=len(key)
        num +=1
    return longstr,posts,ptr

In [4]:
def encodeNumber(n):
    bytes_list = []
    while True:
        bytes_list.insert(0, n % 128)
        if n < 128:
            break
        n = n // 128
    bytes_list[-1] += 128
    return bytearray(bytes_list)
    
def encode(numbers):
    bytes_list = []
    for num in numbers:
        bytes_list.append(encodeNumber(num))
    return b"".join(bytes_list)
    
def decode(bytes_list):
    numbers = []
    n = 0
    for byte in bytes_list:
        if byte < 128:
            n = 128 * n + byte
        else:
            n = 128 * n + (byte - 128)
            numbers.append(n)
            n = 0
    return numbers

In [5]:
def search(t):
    start = 0
    end = 0
    for key in ptr:
        start = ptr[key]
        if(key < len(ptr)-1):
            end = ptr[key+1]
        else:
            end = len(longstr)
        word = longstr[start:end]
        
        if(t == word):
            post = decode(posts[key])           
            temp = [post[0]]
            for i in range(1,len(post),1):
                temp.append(post[i]+temp[i-1])
            return temp
            break

In [6]:
start_time = time.time()
longstr,posts,ptr = compress()
print("time to compress: ", time.time()-start_time)


time to compress:  210.96988773345947


In [ ]:
orgsize = asizeof.asizeof(posting)
strsize = asizeof.asizeof(longstr)
postsize = asizeof.asizeof(posts)
ptrsize = asizeof.asizeof(ptr)

In [ ]:
print("original : ", orgsize/1000000)
print("compressed: ", (strsize+postsize+ptrsize)/1000000)
print("\nLong Str: " , strsize/1000000)
print("Posting size: ", postsize/1000000)
print("Pointer size: ", ptrsize/1000000)

In [ ]:
search("singapor") == posting["singapor"][1]